In [5]:
# Change this as needed
YEAR_TO_PROJECT = 2025

In [6]:
import pandas as pd
import joblib
import os
from helpers import build_results_df
from enum import Enum, auto

In [7]:
class Position(Enum):
    QB = auto()
    RB = auto()
    WR = auto()
    TE = auto()

In [8]:
train_year = YEAR_TO_PROJECT - 1

files_to_load = {
    "df_qb": os.path.join('data', f"{train_year}playerstats_QB.csv"),
    "df_rb_fb": os.path.join('data', f"{train_year}playerstats_RB_FB.csv"),
    "df_wr": os.path.join('data', f"{train_year}playerstats_WR.csv"),
    "df_te": os.path.join('data', f"{train_year}playerstats_TE.csv"),
    "df_rookie_qb": os.path.join('data', f"{YEAR_TO_PROJECT}rookiestats_QB.csv"),
    "df_rookie_rb_fb": os.path.join('data', f"{YEAR_TO_PROJECT}rookiestats_RB_FB.csv"),
    "df_rookie_wr": os.path.join('data', f"{YEAR_TO_PROJECT}rookiestats_WR.csv"),
    "df_rookie_te": os.path.join('data', f"{YEAR_TO_PROJECT}rookiestats_TE.csv"),
    "qb_model": os.path.join('models', "qb_model.joblib"),
    "rb_fb_model": os.path.join('models', "rb_fb_model.joblib"),
    "te_model": os.path.join('models', "te_model.joblib"),
    "wr_model": os.path.join('models', "wr_model.joblib"),
    "rookie_qb_model": os.path.join('models', "rookie_qb_model.joblib"),
    "rookie_rb_fb_model": os.path.join('models', "rookie_rb_fb_model.joblib"),
    "rookie_te_model": os.path.join('models', "rookie_te_model.joblib"),
    "rookie_wr_model": os.path.join('models', "rookie_wr_model.joblib"),
}

missing_files = []
for name, path in files_to_load.items():
    if not os.path.exists(path):
        missing_files.append(path)
    
if missing_files:
    print("Error: The following required files could not be found:")
    for path in missing_files:
        print(f"  - {path}")

    raise FileNotFoundError("Missing required data or model files. Please check the paths.")

In [9]:
dfs = {}
models = {}

for name, path in files_to_load.items():
    if path.endswith('.csv'):
        dfs[name] = pd.read_csv(path)
    elif path.endswith('.joblib'):
        models[name] = joblib.load(path)

In [10]:
from helpers import compute_qb_features, compute_rb_fb_features, compute_wr_features, compute_te_features, compute_rookie_qb_features, compute_rookie_rb_fb_features, compute_rookie_te_features, compute_rookie_wr_features

In [11]:
compute_qb_features(dfs["df_qb"])
compute_rb_fb_features(dfs["df_rb_fb"])
compute_te_features(dfs["df_te"])
compute_wr_features(dfs["df_wr"])
compute_rookie_qb_features(dfs["df_rookie_qb"])
compute_rookie_rb_fb_features(dfs["df_rookie_rb_fb"])
compute_rookie_te_features(dfs["df_rookie_te"])
compute_rookie_wr_features(dfs["df_rookie_wr"])

In [12]:
def generate_projections(model_package, df):
    model = model_package['model']
    scaler = model_package['scaler']
    features = model_package['features']

    data = df[features]
    data_scaled = scaler.transform(data)

    preds = model.predict(data_scaled)

    projections = pd.DataFrame({
        'Player': df['Player'],
        'Projected_Pts': preds
    }, index=df.index)

    return projections

In [13]:
model_to_df_map = {
    "qb_model": "df_qb",
    "rookie_qb_model": "df_rookie_qb",
    "rb_fb_model": "df_rb_fb",
    "rookie_rb_fb_model": "df_rookie_rb_fb",
    "wr_model": "df_wr",
    "rookie_wr_model": "df_rookie_wr",
    "te_model": "df_te",
    "rookie_te_model": "df_rookie_te"
}

all_projections = {}

for model_name, df_name in model_to_df_map.items():
    model_package = models[model_name]
    df = dfs[df_name]
    all_projections[df_name] = generate_projections(model_package, df)

positional_projections = {}
positional_projections[Position.QB] = pd.concat([all_projections['df_qb'], all_projections['df_rookie_qb']], ignore_index=True)
positional_projections[Position.RB] = pd.concat([all_projections['df_rb_fb'], all_projections['df_rookie_rb_fb']], ignore_index=True)
positional_projections[Position.WR] = pd.concat([all_projections['df_wr'], all_projections['df_rookie_wr']], ignore_index=True)
positional_projections[Position.TE] = pd.concat([all_projections['df_te'], all_projections['df_rookie_te']], ignore_index=True)

In [14]:
def compute_vbd(pos: Position, df):
    baseline_ranks = {
        Position.QB: 12,  # Baseline is the 12th best QB
        Position.RB: 24,  # Baseline is the 24th best RB
        Position.WR: 30,  # Baseline is the 30th best WR
        Position.TE: 12,  # Baseline is the 12th best TE
    }

    df.sort_values(by='Projected_Pts', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    baseline_rank = baseline_ranks[pos]

    if baseline_rank > len(df):
        baseline_rank = len(df)
    
    baseline_pts = df.loc[baseline_rank - 1]['Projected_Pts']

    df['VBD'] = df['Projected_Pts'] - baseline_pts


In [15]:
for pos, df in positional_projections.items():
    compute_vbd(pos, df)

final_projections = pd.concat(positional_projections, ignore_index=True)
final_projections.sort_values(by='VBD', ascending=False, inplace=True)
final_projections.reset_index(drop=True, inplace=True)

print(final_projections.to_string())

                       Player  Projected_Pts         VBD
0               Lamar Jackson     360.183217  158.795144
1         Christian McCaffrey     249.027384  109.449611
2                Alvin Kamara     247.631016  108.053242
3              Saquon Barkley     244.298998  104.721224
4               Ja'Marr Chase     267.492439   98.099582
5            Justin Jefferson     249.349130   79.956273
6                  Puka Nacua     245.915306   76.522449
7                Jahmyr Gibbs     213.808691   74.230917
8                Brock Bowers     196.366435   72.418601
9                Trey McBride     195.580721   71.632887
10              George Kittle     194.741001   70.793167
11                 Joe Burrow     270.424311   69.036238
12             Baker Mayfield     267.808239   66.420166
13               Malik Nabers     234.879942   65.487085
14                Breece Hall     200.142797   60.565023
15               Nico Collins     228.366096   58.973239
16                CeeDee Lamb  

In [16]:
os.makedirs('projections', exist_ok=True)
filepath = os.path.join('projections', f'{YEAR_TO_PROJECT}_projections.csv')
final_projections.to_csv(filepath)

print(f"{YEAR_TO_PROJECT} projections has been created.")

2025 projections has been created.
